In [60]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import cirq as cirq
import numpy as np
import matplotlib.pyplot as plt
from xmps.iMPS import iMPS
import unitary_iMPS as qMPS

hello2
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# qmps
This notebook walks through how we represent, optimise, and time evolve uniform matrix product states on a NISQ quantum computer. It depends on xmps, a matrix product state library available at https://github.com/fergusbarratt/xmps, and Cirq, google's quantum programming language

# Intro to uniform Matrix Product States

An uniform matrix product state in right canonical form can be represented by a single repeated tensor

In [2]:
"""
...--A--A--A--A--A--A--...
  |  |  |  |  |  |
""";

Since we can put the above into left canonical form by a gauge transformation, the expectation value of any local observable can be expressed as follows

In [3]:
"""
         --A--...--A--
        |  |       |  |
<O>   = |  ----O----  r
        |  |       |  |
         --A--...--A--

""";

Where r is the largest right eigenvector of the transfer matrix

In [4]:
"""
--A--    ---
  |  |      |
  |  r =    r
  |  |      |
--A--    ---

""";

## Representation

In order to represent a uniform matrix product state of some bond dimension, two objects are sufficient

 - The state tensor A
 - The right eigenmatrix r
 
The main idea of this paper is that these can be represented or determined efficiently on a quantum computer, and the resulting quantum state, representing the state of a infinite, TI quantum system, can be optimized and manipulated in an efficient manner.

### State Tensor

The A tensor of a matrix product state of bond dimension D can be represented on $\log_2(D)+1$ qubits as follows

In [5]:
"""       
            |0>|0>...|0>|0>
               \    j     /
i--A--j  =   |  |   ...  |
   |         -------------
   σ               U
             -------------
             |   ...  |  | 
            /    i     \/σ\
              \log_2(D)  +1
""";

Where we assume that the state unitary $U$ can be efficiently represented on a quantum computer. In the following we denote the depth of $U$ as $p(D)$, where $p$ is subpolynomial, generally we assume constant depth.

In code this is represented by the
`
StateTensor
`
object.

In [104]:
A = iMPS().load('./fixtures/A.npy').left_canonicalise()
A, r = A[0], A.L
V = qMPS.Environment2(qMPS.environment_to_unitary(r))
U = qMPS.StateTensor2(qMPS.left_orthogonal_tensor_to_unitary(A))
S = qMPS.State(U, V)
c = cirq.Circuit().from_ops(S(*cirq.LineQubit.range(3)))
print(c.to_text_diagram(transpose=True))

0 1 2
│ │ │
I V V
|─|─|
U U I
│ │ │


### How to get the environment for a given state tensor

#### Environment

The environment tensor is embedded in a unitary $V$ as follows

In [10]:
"""
            |0>|0>...|0>|0>
i--C--j  =   |  | ... |  |
             -------------
                   v
             -------------
             |  | ... |  | 
            /  i   \/  j  \
           \log_2(D)\log_2(D)
        

r = C@C.conj().T
""";

Where we assume that the environment unitary $V$ can be efficiently represented on a quantum computer. In the following we denote the depth of V as $q(D)$, where $q$ is subpolynomial, generally we assume constant depth.

In code this is represented by the
`
Environment
`
class.

#### Algorithm

Given a StateTensor $U$ (corresponding to $A$), can we efficiently determine the Environment $V$ (corresponding to $r$) on the circuit?

The condition that $VV^\dagger$ is the right environment of U is that $\rho = \sigma$ on the following circuit.

In [9]:
""" 0 0 0 0 0 0 
    | | | | | | 
    | --- | | |       
    |  v  | | |  
    | --- | | |  
    | | | | | |  
    --- | --- |  
     u  |  v  |  
    --- | --- |  
    | | | | | |             
    ρ | | σ | |  

""";

### Tomography

As a first approximation, one can perform state tomography on the $\rho$ and $\sigma$ qubits. This method requires $\sim \mathcal{O}(p(D)3^{\log_2(D)})\sim \mathcal{O}(p(D)D^3)$ operations, but only $3\log_2(D)+1$ qubits. $\mathcal{O}(D^3)$ is the scaling of classical matrix multiplication therefore the scaling of classical DMRG. Can we do better?

### Power Method

Instead of representing the environment compactly, one can represent it as the contraction of $K$ StateTensors, i.e. since it is the largest eigenvector of a matrix, one can use the power method. This is equivalent to approximating the state of the infinite chain by the state of a length $K+1$ uniform chain.

In [ ]:
"""
            |0>|0>...|0>|0>        0 0 0 ... 0     0 0 
i--r--j  =   |  | ... |  |     =   | | | ... |     | |
             -------------         | | | ... ---------
                   v               | | | ...     U     
             -------------         | | | ... --------- 
             |  | ... |  |         | | | ... | ... | | 
            /      \/     \        | | | .........   K
           \log_2(D)\log_2(D)      | | | .........
                                   | | | ... |
                                   | ---------
                                   |     U            
                                   | ---------         
                                   | | ... | | 
                                   | | ... | 2
                                   ---------
                                       U            
                                   ---------           
                                   | ... | | 
                                           1


""";

 - The power method has an error scaling $\epsilon \sim |\frac{\lambda_1}{\lambda_2}|^K$. where $\lambda_i$ is the $i$th largest eigenvalue of the matrix.
 - This method requires $\mathcal{O}(\log_2(D/\epsilon))$ qubits, and requires $p(D)\log(1/\epsilon)$ operations.
 - We can do better than polynomial in $D$! Can we improve the qubit and depth scaling?

### Swap Test

The swap test estimates the overlap $\mathrm{tr}(\rho \sigma)$ between the two states $\rho$ and $\sigma$. Naive implementations require deep circuits, but it can be done in constant depth. 

#### Horizontal 

#### Vertical

### State

All q-local observables of the infinite TI system can be obtained by measurements of corresponding pauli string on the following combination of $U$ and $V$ on the qubits marked $1 \ldots N$

In [11]:
"""
         --A--...--A--
        |  |       |  |
<O>   = |  ----O----  r
        |  |       |  |
         --A--...--A--

      =
        |0>......................|0>
         | | | ... | | ... || ... |
         | | | ... | ---------------
         | | | ... |        V         
         | | | ... | ---------------           
         | | | ... | | ... || ... |
         | | | ... | | ... |
         | | | ... ---------
         | | | ...     U            
         | | | ... ---------            
         | | | ... | ... | | 
         | | | .........   N
         | | | .........
         | | | ... |
         | ---------
         |     U            
         | ---------            
         | | ... | | 
         | | ... | 2
         ---------
             U            
         ---------            
         | ... | | 
                 1
            
""";

and require the creation of a circuit on $(N+1)\log_2(D)$ qubits.

In code the `State` class is the combination of a `StateTensor` and `Environment` object, and takes as an argument $N$.

## Optimization

Classically the state of the art for finding ground states of 1D spin chains is the DMRG algorithm. DMRG is a variational update over matrix product states. Here we outline how to perform the same task on a quantum computer